In [ ]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as f
%matplotlib inline

In [2]:
df=open('names.txt','r').read().splitlines()

In [174]:
words = [
    str(name).strip().strip('"').strip(',').lower() 
    for name in df
    if str(name).strip().strip('"').strip(',').isalpha()
]

In [ ]:
for i in words:
    print(i)

In [ ]:
chars=sorted(set(''.join(words)))
chars

In [6]:
stoi={ s:i+1 for i,s in enumerate(chars)}

In [7]:
stoi['.']=0

In [ ]:
itos={ i:s for s,i in stoi.items()}
itos

In [ ]:
# lets create the test_set
size=3
y=[]
x=[]
for i in words[:5]:
    x_char=[0]*size
    for j in i:
        ch=stoi[j]
        x.append(x_char)
        y.append(ch)
        char=[ itos[c] for c in x_char]
        print(f'{''.join(char)}==> {j}')
        x_char=x_char[1:] + [ch]
       





In [134]:
x=torch.tensor(x,dtype=torch.int64)
y=torch.tensor(y)

In [ ]:
x

In [237]:
look_up_matrix=torch.randn((27,2),requires_grad=True)


In [ ]:
look_up_matrix[x]

In [150]:
emb=look_up_matrix[x]

In [151]:
emb=emb.view(-1,6)

In [241]:
emb.shape

torch.Size([28, 6])

In [ ]:
#

In [140]:
g=torch.Generator().manual_seed(2147483747)
w1=torch.randn((6,100),requires_grad=True,generator=g)
b1=torch.randn((100),requires_grad=True,generator=g)

In [152]:
w2=emb@w1 +b1

In [153]:
h=torch.tanh(w2)
h.shape

torch.Size([28, 100])

In [143]:
final_layer_weight=torch.randn((100,27),requires_grad=True,generator=g)
final_layer_bais=torch.randn((27),requires_grad=True,generator=g)
logits=h@final_layer_weight + final_layer_bais
prob_exp=torch.exp(logits)
prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)



In [154]:
logits=h@final_layer_weight + final_layer_bais
prob_exp=torch.exp(logits)
prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)


In [129]:
logits.shape

torch.Size([28, 27])

tensor(17.7223, grad_fn=<NegBackward0>)

In [155]:
#also rather then above ,we can use
loss=f.cross_entropy(logits,y)
loss

tensor(12.9671, grad_fn=<NllLossBackward0>)

In [156]:
parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]
for p in parameter:
    p.grad=None

In [157]:
loss.backward()


In [147]:
loss.item()

20.869495391845703

In [148]:
for p in parameter:
    p.data+=-.1*p.grad

In [149]:
for p in parameter:
    p.grad=None

In [ ]:
#now the whole code together
#static code we need
y=[]
x=[]
for i in words[:5]:
    x_char=[0]*size
    for j in i:
        ch=stoi[j]
        x.append(x_char)
        y.append(ch)
        char=[ itos[c] for c in x_char]
        x_char=x_char[1:] + [ch]
x=torch.tensor(x,dtype=torch.int64)
y=torch.tensor(y)
g=torch.Generator().manual_seed(2147483747)
look_up_matrix=torch.randn((27,2),requires_grad=True,generator=g)
w1=torch.randn((6,100),requires_grad=True,generator=g)
b1=torch.randn((100),requires_grad=True,generator=g)
final_layer_weight=torch.randn((100,27),requires_grad=True,generator=g)
final_layer_bais=torch.randn((27),requires_grad=True,generator=g)
parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]


In [ ]:
#dynamic code of back_pop
for i in range(100):
    emb=look_up_matrix[x]
    emb=emb.view(-1,6)
    w2=emb@w1 +b1
    h=torch.tanh(w2)
    logits=h@final_layer_weight + final_layer_bais
    #prob_exp=torch.exp(logits)
    #prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)
    loss=f.cross_entropy(logits,y)
    parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]
    for p in parameter:
        p.grad=None
    loss.backward()
    for p in parameter:
        p.data+=-.1*p.grad
    print(loss.item())



In [197]:
def train_set(word):
    x=[]
    y=[]
    for i in words:
        x_char=[0]*size
        for j in i:
            ch=stoi[j]
            x.append(x_char)
            y.append(ch)
            char=[ itos[c] for c in x_char]
            x_char=x_char[1:] + [ch]

    return x,y
    

In [198]:
for_training=int(0.8*len(words))
for_testing=int(0.9*len(words))

In [199]:
x_train,y_train=train_set(words[:for_training])

In [200]:
x_test,y_test=train_set(words[for_training:])

In [ ]:
x_test,y_test

In [285]:
g=torch.Generator().manual_seed(2147483747)
look_up_matrix=torch.randn((27,2),requires_grad=True,generator=g)
w1=torch.randn((6,100),requires_grad=True,generator=g)
b1=torch.randn((100),requires_grad=True,generator=g)
final_layer_weight=torch.randn((100,27),requires_grad=True,generator=g)
final_layer_bais=torch.randn((27),requires_grad=True,generator=g)

In [286]:
def train(x,y,n):
    x=torch.tensor(x,dtype=torch.int64)
    y=torch.tensor(y)
    
    parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]
    #dynamic code of back_prop
    for i in range(n):
        emb=look_up_matrix[x]
        emb=emb.view(-1,6)
        w2=emb@w1 +b1
        h=torch.tanh(w2)
        logits=h@final_layer_weight + final_layer_bais
        #prob_exp=torch.exp(logits)
        #prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)
        loss=f.cross_entropy(logits,y)
        parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]
        for p in parameter:
            p.grad=None
        loss.backward()
        for p in parameter:
            p.data+=-.1*p.grad
    print(loss.item())

In [287]:
def test(x,y):
    x=torch.tensor(x,dtype=torch.int64)
    y=torch.tensor(y)
    emb=look_up_matrix[x]
    emb=emb.view(-1,6)
    w2=emb@w1 +b1
    h=torch.tanh(w2)
    logits=h@final_layer_weight + final_layer_bais
    #prob_exp=torch.exp(logits)
    #prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)
    loss=f.cross_entropy(logits,y)
    parameter=[look_up_matrix,w1,b1,final_layer_weight,final_layer_bais]
    print(loss.item())
   

In [291]:
train(x_train,y_train,100)

2.521152973175049


In [292]:
test(x_test,y_test)

2.520921468734741


In [ ]:
#as we can se test error==train error...our model is underfitting..so lets increase parameters

In [229]:
g2=torch.Generator().manual_seed(2147483747)
look_up_matrix2=torch.randn((27,27),requires_grad=True,generator=g2)
w12=torch.randn((81,200),requires_grad=True,generator=g2)
b12=torch.randn((200),requires_grad=True,generator=g2)
final_layer_weight2=torch.randn((200,27),requires_grad=True,generator=g2)
final_layer_bais2=torch.randn((27),requires_grad=True,generator=g2)

In [279]:
def train2(x,y,n):
    x=torch.tensor(x,dtype=torch.int64)
    y=torch.tensor(y)
    parameter=[look_up_matrix2,w12,b12,final_layer_weight2,final_layer_bais2]
    #dynamic code of back_pop
    for i in range(n):
        emb=look_up_matrix2[x]
        emb=emb.view(-1,81)
        w22=emb@w12 +b12
        h=torch.tanh(w22)
        logits=h@final_layer_weight2 + final_layer_bais2
        #prob_exp=torch.exp(logits)
        #prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)
        loss=f.cross_entropy(logits,y)
        for p in parameter:
            p.grad=None
        loss.backward()
        for p in parameter:
            p.data+=-.1*p.grad
    print(loss.item())

In [280]:
def test2(x,y):
    x=torch.tensor(x,dtype=torch.int64)
    y=torch.tensor(y)
    emb=look_up_matrix2[x]
    emb=emb.view(-1,81)
    w22=emb@w12 +b12
    h=torch.tanh(w22)
    logits=h@final_layer_weight2 + final_layer_bais2
    #prob_exp=torch.exp(logits)
    #prob_exp=prob_exp/torch.sum(prob_exp,1,keepdim=True)
    loss=f.cross_entropy(logits,y)
    parameter=[look_up_matrix2,w12,b12,final_layer_weight2,final_layer_bais2]
    print(loss.item())

In [283]:
train2(x_test,y_test,100)

2.164522409439087


In [284]:
test2(x_train,y_train)

2.164299726486206


In [254]:
#the train2 code took 13 sec,meanwhile the train1 took 2 -3 sec max,this is due to inc in paramters and calcuations but we are far away from underfitting

In [ ]:
#finally the model is in sweet state